In [ ]:
# 5 Hist by water type

# histo by water body type
# import plotly.graph_objects as go

fig = go.Figure()

# Agua subterránea
fig.add_trace(go.Histogram(
    x=df_s.loc[df_s['sub_o_sup']=='Subterránea','fluoruros_tot_float'],
    nbinsx=30,
    name='Agua subterránea',
    opacity=0.6
))

# Agua superficial
fig.add_trace(go.Histogram(
    x=df_s.loc[df_s['sub_o_sup']=='Superficial','fluoruros_tot_float'],
    nbinsx=30,
    name='Agua superficial',
    opacity=0.6
))

# Límite NOM-127
fig.add_vline(
    x=1,
    line_dash="dash",
    line_width=3,
    annotation_text="Límite NOM-127 (1 mg/L)",
    annotation_position="top right"
)

fig.update_layout(
    title="Distribución de Fluoruros Totales: Agua Subterránea vs Superficial",
    xaxis_title="Fluoruros Totales en Muestra de Agua (mg/L)",
    yaxis_title="Frecuencia",
    barmode='overlay'
)

fig.show()


In [ ]:
# 6 Histogram de arsenico con formato distinto

fig = go.Figure()

# Histograma
fig.add_trace(
    go.Histogram(
        x=df_s['as_tot_float'],
        nbinsx=200,
        name="Distribución",
        opacity=0.75
    )
)


# Línea vertical del límite NOM-127 (0.01 mg/L)
fig.add_vline(
    x=0.01,
    line_dash="dash",
    line_color="red",
    line_width=3,
    annotation_text="Límite NOM-127 (0.01 mg/L)",
    annotation_position="top right"
)

# Layout mejorado
fig.update_layout(
    title="Distribución de Arsénico Total en Muestras de Agua (México)",
    xaxis_title="Arsénico Total en Muestra de Agua (mg/L)",
    yaxis_title="Frecuencia",
    bargap=0.05
)

fig.show()



In [ ]:
# Se crea una figura vacía y luego se añade un rastro de scatter
# fig = go.Figure(data=[go.Scatter(x=car_data['odometer'], y=car_data['price'], mode='markers')])

In [ ]:
# Filtrar observaciones válidas
df_plot = df_s.dropna(subset=['as_tot_float', 'fecha_realizacion_dt', 'sub_o_sup'])

fig = px.scatter(
    df_plot,
    x='fecha_realizacion_dt',
    y='as_tot_float',
    color='sub_o_sup',
    title='Dispersión de Arsénico en Muestras de Agua a lo Largo del Tiempo Por Cuerpo de Agua 2012-20214',
    labels={
        'fecha_realizacion_dt': 'Fecha de Realización de la Muestra',
        'as_tot_float': 'Arsénico Total (mg/L)',
        'sub_o_sup': 'Tipo de Cuerpo de Agua'
    }
)

fig.update_layout(
    legend_title_text='Tipo de Cuerpo de Agua',
    yaxis=dict(rangemode='tozero')
)

fig.add_hline(
    y=0.01,
    line_dash="dash",
    line_color="red",
    annotation_text="Límite NOM-127 (0.01 mg/L)"
)


# Mostrar el gráfico Plotly
fig.show()


In [ ]:
# express grafico en version de plotly.graph_objects, hacer un grafico por tipo de agua sin usar subplots


import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filtrar datos válidos
df_plot = df_s.dropna(subset=['as_tot_float', 'fecha_realizacion_dt', 'sub_o_sup'])

# Obtener tipos de agua presentes
tipos = df_plot['sub_o_sup'].unique()

# Crear subplots
fig = make_subplots(
    rows=1,
    cols=len(tipos),
    subplot_titles=[f"Fuente: {t}" for t in tipos],
    shared_xaxes=False,
    shared_yaxes=False
)

# Añadir scatter por tipo de agua
for i, t in enumerate(tipos, start=1):
    df_temp = df_plot[df_plot['sub_o_sup'] == t]

    fig.add_trace(
        go.Scatter(
            x=df_temp['fecha_realizacion_dt'],
            y=df_temp['as_tot_float'],
            mode='markers',
            name=t,
            showlegend=True
        ),
        row=1,
        col=i
    )

    # Limitar eje Y
    fig.update_yaxes(range=[0, 0.2], row=1, col=i)

    # Línea NOM-127
    fig.add_hline(
        y=0.01,
        line_dash="dash",
        line_color="red",
        line_width=3,
        annotation_text="Límite NOM-127 (0.01 mg/L)",
        row=1,
        col=i
    )

# Layout general
fig.update_layout(
    title="Dispersión de Arsénico en Agua por Tipo de Fuente (México)",
    xaxis_title="Fecha de Realización de la Muestra",
    yaxis_title="Arsénico Total (mg/L)",
    legend_title="Tipo de Fuente de Agua"
)

fig.show()


In [ ]:
quiero crear la misma tabla de proporcion de excesos pero por tipo de agua, es decir, la proporcion de muestras que exceden el limite de arsenico total por tipo de agua y por ano de 2012 a 2024
===
tabla_excesos = (
    df_s
    .dropna(subset=['as_tot_float'])
    .groupby('sub_o_sup')
    .agg(
        total=('as_tot_float', 'count'),
        exceden=('supNOM127SSA12021_As', lambda x: (x == 1).sum())
    )
)

tabla_excesos['proporcion_exceso'] = (
    tabla_excesos['exceden'] /
    tabla_excesos['total']
)

print(tabla_excesos)
===

In [ ]:
import plotly.graph_objects as go

# Filtrar datos válidos primero
df_plot = df_s.dropna(subset=['as_tot_float', 'profundidad_float', 'sub_o_sup'])

# Separar datasets
df_sub = df_plot[df_plot['sub_o_sup'] == 'Subterránea']
df_sup = df_plot[df_plot['sub_o_sup'] == 'Superficial']


# ---------- AGUA SUBTERRÁNEA ----------
fig_sub = go.Figure()

fig_sub.add_trace(
    go.Scatter(
        x=df_sub['profundidad_float'],
        y=df_sub['as_tot_float'],
        mode='markers',
        name='Agua subterránea',
        marker=dict(size=5, opacity=0.6)
    )
)

fig_sub.update_layout(
    title='Arsénico vs Profundidad en Agua Subterránea (México)',
    xaxis_title='Profundidad del Sitio (m)',
    yaxis_title='Arsénico Total (mg/L)'
)

fig_sub.show()


# ---------- AGUA SUPERFICIAL ----------
fig_sup = go.Figure()

fig_sup.add_trace(
    go.Scatter(
        x=df_sup['profundidad_float'],
        y=df_sup['as_tot_float'],
        mode='markers',
        name='Agua superficial',
        marker=dict(size=5, opacity=0.6)
    )
)

fig_sup.update_layout(
    title='Arsénico vs Profundidad en Agua Superficial (México)',
    xaxis_title='Profundidad del Sitio (m)',
    yaxis_title='Arsénico Total (mg/L)'
)

fig_sub.add_hline(y=0.01, line_dash="dash", line_color="red")
fig_sup.add_hline(y=0.01, line_dash="dash", line_color="red")

fig_sup.show()


In [ ]:
# 13 Grafico de dispersion - as totales por profundidad en agua subterranea vs superficial
# Crear un scatter plot utilizando plotly.graph_objects

print(df_s['profundidad_float'].describe())

In [ ]:


# Filtrar solo aguas superficiales con datos válidos
df_sup = (
    df_s
    .query("sub_o_sup == 'Superficial'")
    .dropna(subset=['as_tot_float', 'profundidad_float'])
    .copy()
)

# Categorías sustantivas
bins = [0, 5, 20, 50, 120]
labels = [
    "0–5 m (somera)",
    "5–20 m (intermedia)",
    "20–50 m (profunda)",
    "50–120 m (muy profunda)"
]

df_sup['profundidad_nivel'] = pd.cut(
    df_sup['profundidad_float'],
    bins=bins,
    labels=labels,
    right=False,
    include_lowest=True
)

# Tabla de excedencias NOM-127
tabla_excesos_prof = (
    df_sup
    .dropna(subset=['profundidad_nivel'])
    .groupby('profundidad_nivel')
    .agg(
        total=('as_tot_float', 'count'),
        exceden=('supNOM127SSA12021_As', lambda x: (x == 1).sum())
    )
)

tabla_excesos_prof['proporcion_exceso'] = (
    tabla_excesos_prof['exceden'] /
    tabla_excesos_prof['total']
)

# Formatear proporciones a %
tabla_excesos_prof['proporcion_exceso'] = (
    tabla_excesos_prof['proporcion_exceso']
    .apply(lambda x: f"{x:.2%}" if pd.notna(x) else x)
)

print(tabla_excesos_prof)

# 0  depth measures for ground water oddly enough


the version of python associated with the selected kernel is no lnger supported. Please consider selecting a different kernel

In [ ]:
# explore depth by water type 
print(df_s['profundidad_float'].describe())
print()
print(df_s.groupby("sub_o_sup")["profundidad_float"].describe())

In [ ]:
# arsenico

print('exceso de fluoruros totales por tipo de agua')
tabla_excesos_f = (
    df_s
    .dropna(subset=['fluoruros_tot_float', 'tipo_cuerpo_simplificado'])
    .groupby('tipo_cuerpo_simplificado')
    .agg(
        total=('fluoruros_tot_float', 'count'),
        exceden=('supNOM127SSA12021_F', lambda x: (x == 1).sum())
    )
)

tabla_excesos_f['proporcion_exceso'] = tabla_excesos_f['exceden'] / tabla_excesos_f['total']

# Formato % con 2 decimales
tabla_excesos_f['proporcion_exceso'] = tabla_excesos_f['proporcion_exceso'].apply(
    lambda x: f"{x:.2%}" if pd.notna(x) else x
)

print(tabla_excesos_f)
print()

# fluor 

print('exceso de arsenico total por tipo de agua')
tabla_excesos_f = (
    df_s
    .dropna(subset=['as_tot_float', 'tipo_cuerpo_simplificado'])
    .groupby('tipo_cuerpo_simplificado')
    .agg(
        total=('as_tot_float', 'count'),
        exceden=('supNOM127SSA12021_As', lambda x: (x == 1).sum())
    )
)

tabla_excesos_f['proporcion_exceso'] = tabla_excesos_f['exceden'] / tabla_excesos_f['total']

# Formato % con 2 decimales
tabla_excesos_f['proporcion_exceso'] = tabla_excesos_f['proporcion_exceso'].apply(
    lambda x: f"{x:.2%}" if pd.notna(x) else x
)

print(tabla_excesos_f)


In [ ]:
# fluoruros 

import pandas as pd
import numpy as np

# 1) Subset solo variables relevantes
df_f = (
    df_s
    .dropna(subset=['fluoruros_tot_float', 'profundidad_float', 'sub_o_sup'])
    .copy()
)

# 2) Niveles sustantivos de profundidad (ajustados a tu rango 0.02–120 m)
bins = [0, 5, 20, 50, 120]
labels = [
    "0–5 m (somera)",
    "5–20 m (intermedia)",
    "20–50 m (profunda)",
    "50–120 m (muy profunda)"
]

df_f['profundidad_nivel'] = pd.cut(
    df_f['profundidad_float'],
    bins=bins,
    labels=labels,
    right=False,
    include_lowest=True
)

# 3) Tabla base
tabla = (
    df_f
    .dropna(subset=['profundidad_nivel'])
    .groupby(['profundidad_nivel', 'sub_o_sup'])
    .agg(
        total=('fluoruros_tot_float', 'count'),
        exceden=('supNOM127SSA12021_F', lambda x: (x == 1).sum())
    )
)

tabla['proporcion_exceso'] = tabla['exceden'] / tabla['total']

# 4) Pivot
pivot_pct = tabla['proporcion_exceso'].unstack('sub_o_sup')
pivot_total = tabla['total'].unstack('sub_o_sup')

# 5) Formatear porcentajes para visualización
pivot_pct_fmt = pivot_pct.applymap(lambda x: f"{x:.2%}" if pd.notna(x) else np.nan)

print("=== % Excedencias Fluoruros NOM-127 por profundidad y tipo de agua ===")
print(pivot_pct_fmt)

print("\n=== Totales de muestras ===")
print(pivot_total)

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# -----------------------------
# FLUORUROS
# -----------------------------
tabla_fluor = (
    df_s
    .dropna(subset=['fluoruros_tot_float', 'tipo_cuerpo_simplificado'])
    .groupby('tipo_cuerpo_simplificado')
    .agg(
        total=('fluoruros_tot_float', 'count'),
        exceden=('supNOM127SSA12021_F', lambda x: (x == 1).sum())
    )
)

tabla_fluor['proporcion_exceso'] = tabla_fluor['exceden'] / tabla_fluor['total']

fig_fluor = go.Figure()

fig_fluor.add_trace(
    go.Bar(
        x=tabla_fluor.index,
        y=tabla_fluor['proporcion_exceso'] * 100,
        text=[f"{v:.2%}" for v in tabla_fluor['proporcion_exceso']],
        textposition='outside',
        name='Fluoruros'
    )
)

fig_fluor.update_layout(
    title="Porcentaje de Excedencias de Fluoruros 1mg/L (NOM-127-SSA1-2021) por Tipo de Agua",
    xaxis_title="Tipo de Cuerpo de Agua",
    yaxis_title="% de muestras que exceden límite",
    yaxis=dict(range=[0, 100])
)

text=[f"{p:.2%}<br>(n={n})" 
      for p, n in zip(tabla_fluor['proporcion_exceso'], tabla_fluor['total'])]

fig_fluor.update_traces(text=text)
fig_fluor.show()


# -----------------------------
# ARSÉNICO
# -----------------------------
tabla_as = (
    df_s
    .dropna(subset=['as_tot_float', 'tipo_cuerpo_simplificado'])
    .groupby('tipo_cuerpo_simplificado')
    .agg(
        total=('as_tot_float', 'count'),
        exceden=('supNOM127SSA12021_As', lambda x: (x == 1).sum())
    )
)

tabla_as['proporcion_exceso'] = tabla_as['exceden'] / tabla_as['total']

fig_as = go.Figure()

fig_as.add_trace(
    go.Bar(
        x=tabla_as.index,
        y=tabla_as['proporcion_exceso'] * 100,
        text=[f"{v:.2%}" for v in tabla_as['proporcion_exceso']],
        textposition='outside',
        name='Arsénico'
    )
)

fig_as.update_layout(
    title="Porcentaje de Excedencias de Arsénico 0.01 mg/L (NOM-127-SSA1-2021) por Tipo de Agua",
    xaxis_title="Tipo de Cuerpo de Agua",
    yaxis_title="% de muestras que exceden límite",
    yaxis=dict(range=[0, 100])
)

text=[f"{p:.2%}<br>(n={n})" 
      for p, n in zip(tabla_as['proporcion_exceso'], tabla_as['total'])]

fig_as.update_traces(text=text)

fig_as.show()


In [ ]:
###########
# PASO 4
###########


# Script del cuadro de mandos de la aplicacion web

# load libraries

import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# load data
df_s = pd.read_parquet('../tlm_conagua_limpio.parquet')

########################################################################
########################################################################
# Crear contendio de la apli basada en Streamlit
########################################################################
########################################################################


########################################################################
# Crear un encabezado
########################################################################

st.header('Visualizaciones de Niveles de Fluoruros y Arsenico en Los Cuerpos de Agua de Mexico de 2012-2024 con Datos de CONAGUA')

########################################################################
# Crear 1 boton para construir el histograma de arsenico
########################################################################

hist_button1 = st.button('Construir histograma de arsenico')

# Logica a ejectuar al hacer click en el boton
if hist_button1:
    # escribir un menasaje en la apli
    st.write('Creacion de un histograma de los niveles de arsenico total de 2012-2024')

    # crear un hist con plotly.graph_objects
    fig = go.Figure()

    # Histograma
    fig.add_trace(
        go.Histogram(
            x=df_s['as_tot_float'],
            xbins=dict(
                start=0,
                end=1,
                size=0.005   # thinner bins
            ),
            name="Distribución",
            opacity=0.75
        )
    )
    # Línea vertical del límite NOM-127 (0.01 mg/L)
    fig.add_vline(
        x=0.01,
        line_dash="dash",
        line_color="red",
        line_width=3,
        annotation_text="Límite NOM-127 (0.01 mg/L)",
        annotation_position="top right"
    )
    # Layout mejorado
    fig.update_layout(
        title="Distribución de Arsénico Totales en Muestras de Agua (México)",
        xaxis_title="Arsénico Total en Muestra de Agua (mg/L)",
        yaxis_title="Frecuencia",
        bargap=0.05,
        # ← clave para limitar el rango del eje x a 0-0.2 mg/L
        xaxis=dict(range=[0, 0.2])
    )
    # mostrar el grafico interactivo en la apli streamlit
    st.plotly_chart(fig, use_container_width=True)

########################################################################
# Crear 2 otro boton para crear el histograma de fluoruros
########################################################################

hist_button2 = st.button('Construir histograma de fluoruros')

# Logica a ejectuar al hacer click en el boton
if hist_button2:
    # escribir un menasaje en la apli
    st.write(
        'Creacion de un histograma de los niveles de fluoruros totales de 2012-2024')
    # crear un hist con plotly.graph_objects
    fig = go.Figure()
    fig.add_trace(
        go.Histogram(
            x=df_s['fluoruros_tot_float'],
            nbinsx=30,
            name="Distribución",
            opacity=0.75
        )
    )
    # Línea vertical del límite NOM-127 (1 mg/L)
    fig.add_vline(
        x=1,
        line_dash="dash",
        line_color="red",
        line_width=3,
        annotation_text="Límite NOM-127 (1 mg/L)",
        annotation_position="top right"
    )
    # Layout mejorado
    fig.update_layout(
        title="Distribución de Fluoruros Totales en Muestras de Agua (México)",
        xaxis_title="Fluoruros Totales en Muestra de Agua (mg/L)",
        yaxis_title="Frecuencia",
        bargap=0.05
    )
    # mostra el grafico interactivo en la apli
    st.plotly_chart(fig, use_container_width=True)

########################################################################
# Crear 3 otro botton para construit un grafico de dispersion con plotly
########################################################################

disp_button = st.button(
    'Construir grafico de dispersion de arsenico en el tiempo')

# logica a ejecutar cuando se hace clic en el boton
if disp_button:
    # escribir un mensaje
    st.write('Creacion de un grafico de dispersion de los mg/L de arsenico encontrado en las muestras de agua atraves del tiempo segun el tipo de agua')

    # Filtrar datos válidos
    df_plot = df_s.dropna(
        subset=['as_tot_float', 'fecha_realizacion_dt', 'sub_o_sup'])

    fig = px.scatter(
        df_plot,
        x='fecha_realizacion_dt',
        y='as_tot_float',
        color='sub_o_sup',
        facet_col='sub_o_sup',
        opacity=0.6,
        title='Dispersión de Arsénico en Agua por Tipo de Cuerpo de Agua (México)',
        labels={
            'fecha_realizacion_dt': 'Fecha de Realización de la Muestra',
            'as_tot_float': 'Arsénico Total (mg/L)',
            'sub_o_sup': 'Tipo de Cuerpo de Agua'
        }
    )
    # Limitar eje Y para mejor visibilidad
    fig.update_yaxes(range=[0, .2], title_text='Arsénico Total (mg/L)')

    # Línea NOM-127 en cada panel
    fig.add_hline(
        y=0.01,
        line_dash="dash",
        line_color="red",
        line_width=3,
        annotation_text="Límite NOM-127 (0.01 mg/L)"
    )
    fig.update_layout(legend_title_text='Tipo de Agua')

    st.plotly_chart(fig, use_container_width=True)


In [ ]:
# PASO 4

########################################################################
# Script del cuadro de mandos de la aplicacion web
########################################################################

# load libraries

import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# load data
df_s = pd.read_parquet('../tlm_conagua_limpio.parquet')


########################################################################
# Crear contendio de la apli basada en Streamlit
########################################################################

########################################################################
# Crear un encabezado
########################################################################

st.header('Visualizaciones de Niveles de Fluoruros y Arsenico en Los Cuerpos de Agua de Mexico de 2012-2024 con Datos de CONAGUA')

########################################################################
# Crear casillas de verificacion
#######################################################################

st.subheader('Selecciona los graficos que deseas visualizar:')

# Casilla 1 : hist as_tot
hist_As = st.checkbox('Construir un histograma de arsenico')

# Casilla 2 : hist fluoruros_tot
hist_F = st.checkbox('Construir un histograma de fluoruros')

# Casilla 3 : scatter as_tot + fecha_realizacion_dt
scatter_As = st.checkbox(
    'Construir un grafico de dispersion de arsencio en el tiempo')

# Casilla 4 : barras as_tot + ano + sub_o_sup
bar_As = st.checkbox(
    'Construir un grafico de barras de excesso arsencio en el tiempo')

########################################################################
# Logica para mostrar graficos segun la casilla seleccionada
########################################################################

# grafico 1
if hist_As:
    # escribir un menasaje en la apli
    st.write('Creacion de un histograma de los niveles de arsenico total de 2012-2024')

    # crear un hist con plotly.graph_objects
    fig = go.Figure()
    fig.add_trace(
        go.Histogram(
            x=df_s['as_tot_float'],
            xbins=dict(
                start=0,
                end=1,
                size=0.005   # thinner bins
            ),
            name="Distribución",
            opacity=0.75
        )
    )
    # Línea vertical del límite NOM-127 (0.01 mg/L)
    fig.add_vline(
        x=0.01,
        line_dash="dash",
        line_color="red",
        line_width=3,
        annotation_text="Límite NOM-127 (0.01 mg/L)",
        annotation_position="top right"
    )
    # Layout mejorado
    fig.update_layout(
        title="Distribución de Arsénico Totales en Muestras de Agua (México)",
        xaxis_title="Arsénico Total en Muestra de Agua (mg/L)",
        yaxis_title="Frecuencia",
        bargap=0.05,
        # ← clave para limitar el rango del eje x a 0-0.2 mg/L
        xaxis=dict(range=[0, 0.2])
    )
    # mostrar el grafico interactivo en la apli streamlit
    st.plotly_chart(fig, use_container_width=True)


# grafico 2
if hist_F:
    # escribir un menasaje en la apli
    st.write(
        'Creacion de un histograma de los niveles de fluoruros totales de 2012-2024')
    # crear un hist con plotly.graph_objects
    fig = go.Figure()
    fig.add_trace(
        go.Histogram(
            x=df_s['fluoruros_tot_float'],
            nbinsx=30,
            name="Distribución",
            opacity=0.75
        )
    )
    # Línea vertical del límite NOM-127 (1 mg/L)
    fig.add_vline(
        x=1,
        line_dash="dash",
        line_color="red",
        line_width=3,
        annotation_text="Límite NOM-127 (1 mg/L)",
        annotation_position="top right"
    )
    # Layout mejorado
    fig.update_layout(
        title="Distribución de Fluoruros Totales en Muestras de Agua (México)",
        xaxis_title="Fluoruros Totales en Muestra de Agua (mg/L)",
        yaxis_title="Frecuencia",
        bargap=0.05
    )
    # mostra el grafico interactivo en la apli
    st.plotly_chart(fig, use_container_width=True)


# grafico 3
if scatter_As:
    # escribir un mensaje
    st.write('Creacion de un grafico de dispersion de los mg/L de arsenico encontrado en las muestras de agua atraves del tiempo segun el tipo de agua')
    # filtrar datos válidos
    df_plot = df_s.dropna(
        subset=['as_tot_float', 'fecha_realizacion_dt', 'sub_o_sup'])
    # plot
    fig = px.scatter(
        df_plot,
        x='fecha_realizacion_dt',
        y='as_tot_float',
        color='sub_o_sup',
        facet_col='sub_o_sup',
        opacity=0.6,
        title='Dispersión de Arsénico en Agua por Tipo de Cuerpo de Agua (México)',
        labels={
            'fecha_realizacion_dt': 'Fecha de Realización de la Muestra',
            'as_tot_float': 'Arsénico Total (mg/L)',
            'sub_o_sup': 'Tipo de Cuerpo de Agua'
        }
    )
    # Limitar eje Y para mejor visibilidad
    fig.update_yaxes(range=[0, .2], title_text='Arsénico Total (mg/L)')
    # Línea NOM-127 en cada panel
    fig.add_hline(
        y=0.01,
        line_dash="dash",
        line_color="red",
        line_width=3,
        annotation_text="Límite NOM-127 (0.01 mg/L)"
    )
    fig.update_layout(legend_title_text='Tipo de Agua')
    # mostrar el grafico en la apli
    st.plotly_chart(fig, use_container_width=True)


# grafico 4

if bar_As:
    # escribir un mensaje
    st.write('Creacion de un grafico de barra de los excesos de arsenico encontrados anualmente en aguas subterraneas y superficiales')
    # 1) Crear tabla anual (numérica) si no la tienes ya
    tabla_excesos_anual = (
        df_s
        .dropna(subset=['as_tot_float'])
        .query("2012 <= ano <= 2024")
        .groupby(['ano', 'sub_o_sup'])
        .agg(
            total=('as_tot_float', 'count'),
            exceden=('supNOM127SSA12021_As', lambda x: (x == 1).sum())
        )
    )
    tabla_excesos_anual['proporcion_exceso'] = (
        tabla_excesos_anual['exceden'] / tabla_excesos_anual['total']
    )
    # 2) Pivot numérico (MultiIndex en columnas)
    pivot = (
        tabla_excesos_anual[['total', 'proporcion_exceso']]
        .unstack('sub_o_sup')
    )
    # 3) Helper para construir un subplot por tipo

    def add_bar(fig, col_idx, tipo, titulo_subplot):
        # Series por año
        y_prop = pivot[('proporcion_exceso', tipo)] * 100          # a %
        txt_tot = pivot[('total', tipo)].astype(
            'Int64')           # totales (puede tener <NA>)
        # limpiar NaNs para plot
        years = y_prop.index
        y_vals = y_prop.values
        txt_vals = [f"n={t}" if pd.notna(t) else "" for t in txt_tot.values]
        fig.add_trace(
            go.Bar(
                x=years,
                y=y_vals,
                name=tipo,
                text=txt_vals,          # texto arriba de la barra
                textposition="outside"
            ),
            row=1,
            col=col_idx
        )
        fig.update_xaxes(title_text="Año", row=1, col=col_idx)
        fig.update_yaxes(title_text="% que excede 0.01 mg/L",
                         row=1, col=col_idx, rangemode="tozero")
        # título del subplot
        fig.layout.annotations[col_idx-1].text = titulo_subplot
    # 4) Definir qué tipos existen en tus datos (ajusta si tus etiquetas son distintas)
    tipos_disponibles = list(pivot.columns.get_level_values(1).unique())
    tipo_sub = 'Subterránea'
    tipo_sup = 'Superficial'
    # 5) Crear figura con 2 subplots (uno por tipo)
    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=("Agua subterránea", "Agua superficial"),
        shared_yaxes=False
    )
    # Agregar barras (si existen)
    if ('proporcion_exceso', tipo_sub) in pivot.columns:
        add_bar(fig, 1, tipo_sub, "Agua subterránea")
    else:
        print(
            f"⚠️ No encontré el tipo '{tipo_sub}' en sub_o_sup. Tipos disponibles: {tipos_disponibles}")

    if ('proporcion_exceso', tipo_sup) in pivot.columns:
        add_bar(fig, 2, tipo_sup, "Agua superficial")
    else:
        print(
            f"⚠️ No encontré el tipo '{tipo_sup}' en sub_o_sup. Tipos disponibles: {tipos_disponibles}")
    # 6) Layout general
    fig.update_layout(
        title="Porcentaje de muestras anuales que exceden el límite de Arsénico (NOM-127-SSA1-2021)",
        showlegend=False,
        bargap=0.15
    )
    # 7) mostrar el grafico en la apli
    st.plotly_chart(fig, use_container_width=True)
